In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import subprocess
import re


def shmoogle_smi():
    jax.profiler.save_device_memory_profile("memory.prof")
    pprof_path = "/usr/local/go/pkg/tool/linux_amd64/pprof"
    out = subprocess.run([pprof_path, "-top", "memory.prof"], stdout=subprocess.PIPE)
    stdout = out.stdout.decode("utf-8")
    re_sult = re.search(
        r"Showing nodes accounting for (\d+(?:\.\d+)?)([MG]B)?, (\d+(?:\.\d+)?)% of (\d+(?:\.\d+)?)([MG]B)? total",
        stdout,
    )
    multiplier = 1 / 1000 if re_sult.group(5) == "MB" else 1
    total_mem_usage = float(re_sult.group(4))
    print(
        "Total mem usage:",
        total_mem_usage * multiplier,
        "GB",
        "out of",
        16 * len(jax.devices()),
        "GB",
    )


shmoogle_smi()

Total mem usage: 0.0 GB out of 128 GB


In [3]:
from tokenizer import Tokenizer
from llama2_model import LLaMA

import equinox as eqx
import re
from safetensors import safe_open
from jax.sharding import Mesh, NamedSharding, PartitionSpec
import jax.numpy as jnp
import transformers
import numpy as np
import jax
import jmp

/home/neverix/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = Tokenizer("models/Llama-2-7b-hf/tokenizer.model")
input_ids = tokenizer.encode("Hello world!", bos=True, eos=False)
input_ids

[1, 15043, 3186, 29991]

In [5]:
num_devices = len(jax.devices())
mesh = Mesh(np.array(jax.devices()).reshape(-1, 4), axis_names=("dp", "mp"))
policy = jmp.get_policy("p=bf16,c=bf16")
print("Creating LLaMA...")
llama = LLaMA(mesh, policy)
print("Created LLaMA.")
shmoogle_smi()

Creating LLaMA...
Created LLaMA.
Total mem usage: 30.0 GB out of 128 GB


Main binary filename not available.


In [7]:
print("Loading model...")
print()
for filename in [
    "models/Llama-2-7b-hf/model-00001-of-00002.safetensors",
    "models/Llama-2-7b-hf/model-00002-of-00002.safetensors",
]:
    with safe_open(
        filename,
        framework="numpy",
        device="cpu",
    ) as f:
        for k in f.keys():
            weight = f.get_tensor(k)
            if (
                k.endswith(".weight")
                and not k.endswith("embed_tokens.weight")
                and not k.endswith("norm.weight")
                # and not k.endswith("lm_head.weight")
            ):
                weight = weight.T
            re_sult = re.search(r"layers\.([0-9]+)", k)
            try:
                k = (
                    k[: re_sult.span()[0]]
                    + f"layers[{re_sult.group(1)}]"
                    + k[re_sult.span()[1] :]
                )
            except AttributeError:
                pass
            print("\r" + " " * 80, end="")
            print("\rLoading", k, end="")
            og = eval(f"llama.{k}")
            weight = jax.device_put(weight.astype(og.dtype), device=og.sharding)
            llama = eval(f"eqx.tree_at(lambda l: l.{k}, llama, weight)")
print()
shmoogle_smi()

Loading model...

Loading model.embed_tokens.weight                                               

Loading model.norm.weight                                                       
Total mem usage: 34.41 GB out of 128 GB


Main binary filename not available.


In [36]:
from llama2_model import SelfAttention
from ast import (
    NodeTransformer,
    Expr,
    Return,
    Subscript,
    NamedExpr,
    Name,
    Store,
    Load,
    Constant,
    Index,
    FunctionDef,
    Call,
    Attribute,
    arg,
    arguments,
    Assign,
    Tuple,
    Dict,
    keyword,
    IfExp,
    Lambda,
)
import inspect
import ast
import dis


class ReplaceFuncCall(NodeTransformer):
    def visit_Call(self, node):
        func = node.func
        store = Call(
            func=Name(id="getattr", ctx=Load()),
            args=[
                func,
                Constant(value="store", kind=None),
                Lambda(
                    args=arguments(
                        posonlyargs=[],
                        args=[
                            arg(arg="state", annotation=None, type_comment=None),
                            arg(arg="output", annotation=None, type_comment=None),
                        ],
                        vararg=None,
                        kwonlyargs=[],
                        kw_defaults=[],
                        kwarg=None,
                        defaults=[],
                    ),
                    body=Tuple(
                        elts=[
                            Name(id="output", ctx=Load()),
                            Name(id="state", ctx=Load()),
                        ],
                        ctx=Load(),
                    ),
                ),
            ],
            keywords=[],
        )
        return Subscript(
            value=NamedExpr(
                target=Name(id="state", ctx=Store()),
                value=Call(
                    func=store,
                    args=[
                        Name(id="state", ctx=Load()),
                        Call(
                            func=func,
                            args=[self.visit(a) for a in node.args],
                            keywords=[self.visit(k) for k in node.keywords]
                            + [
                                keyword(
                                    arg=None,
                                    value=IfExp(
                                        test=Call(
                                            func=Name(id="hasattr", ctx=Load()),
                                            args=[
                                                func,
                                                Constant(value="store", kind=None),
                                            ],
                                            keywords=[],
                                        ),
                                        body=Dict(
                                            keys=[Constant(value="state", kind=None)],
                                            values=[Name(id="state", ctx=Load())],
                                        ),
                                        orelse=Dict(keys=[], values=[]),
                                    ),
                                )
                            ],
                        ),
                    ],
                    keywords=[],
                ),
            ),
            slice=Index(value=Constant(value=0, kind=None)),
            ctx=Load(),
        )
        return

    def visit_FunctionDef(self, node):
        return FunctionDef(
            name=node.name,
            args=arguments(
                args=node.args.args,
                vararg=node.args.vararg,
                kwonlyargs=node.args.kwonlyargs
                + [arg(arg="state", annotation=None, type_comment=None)],
                kw_defaults=node.args.kw_defaults + [Dict(keys=[], values=[])],
                kwarg=node.args.kwarg,
                defaults=node.args.defaults,
                posonlyargs=node.args.posonlyargs,
            ),
            body=
            # [
            #     Assign(
            #         targets=[Name(id="state", ctx=Store())],
            #         value=Tuple(
            #             elts=[
            #                 Constant(value=None, kind=None),
            #                 Dict(keys=[], values=[]),
            #             ],
            #             ctx=Load(),
            #         ),
            #         type_comment=None,
            #     )
            # ]
            # +
            [self.visit(b) for b in node.body],
            decorator_list=node.decorator_list,
            returns=node.returns,
            type_comment=node.type_comment,
        )

    def visit_Return(self, node: Return):
        return Return(
            value=Tuple(
                elts=[self.visit(node.value), Name(id="state", ctx=Load())],
                ctx=Load(),
            )
        )


def add_state(func):
    source = inspect.getsource(func)
    if source.startswith(" "):
        source = "if True:\n" + source
    func_ast = ast.parse(source)
    func_ast = ReplaceFuncCall().visit(func_ast)
    func_ast = ast.fix_missing_locations(func_ast)
    print(ast.dump(func_ast))
    code = compile(func_ast, filename=func.__code__.co_filename, mode="exec")
    env = func.__globals__.copy()
    import astor

    print(astor.to_source(func_ast))
    print(env.keys())
    exec(code, env)
    return env[func.__name__]


def blabla(a, b):
    return a - b


blabla.store = lambda state, output: (output, {"a": 1, **state})
add_state(SelfAttention.__call__)

Module(body=[If(test=Constant(value=True, kind=None), body=[FunctionDef(name='__call__', args=arguments(posonlyargs=[], args=[arg(arg='self', annotation=None, type_comment=None), arg(arg='x', annotation=None, type_comment=None), arg(arg='attention_mask', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[arg(arg='state', annotation=None, type_comment=None)], kw_defaults=[Dict(keys=[], values=[])], kwarg=None, defaults=[Constant(value=None, kind=None)]), body=[Assign(targets=[Name(id='x', ctx=Store())], value=Subscript(value=NamedExpr(target=Name(id='state', ctx=Store()), value=Call(func=Call(func=Name(id='getattr', ctx=Load()), args=[Attribute(value=Attribute(value=Name(id='self', ctx=Load()), attr='policy', ctx=Load()), attr='cast_to_compute', ctx=Load()), Constant(value='store', kind=None), Lambda(args=arguments(posonlyargs=[], args=[arg(arg='state', annotation=None, type_comment=None), arg(arg='output', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw

<function llama2_model.__call__(self, x, attention_mask=None, *, state={})>

In [32]:
ast.dump(ast.parse("f(a, b=2, **({'state': state} if hasattr(f, 'store') else {}))"))

"Module(body=[Expr(value=Call(func=Name(id='f', ctx=Load()), args=[Name(id='a', ctx=Load())], keywords=[keyword(arg='b', value=Constant(value=2, kind=None)), keyword(arg=None, value=IfExp(test=Call(func=Name(id='hasattr', ctx=Load()), args=[Name(id='f', ctx=Load()), Constant(value='store', kind=None)], keywords=[]), body=Dict(keys=[Constant(value='state', kind=None)], values=[Name(id='state', ctx=Load())]), orelse=Dict(keys=[], values=[])))]))], type_ignores=[])"

In [42]:
class DebugWrapper(eqx.Module):
    name: str
    module: eqx.Module
    write_to: dict

    def __init__(self, name, module):
        self.name = name
        self.module = module

    def __call__(self, *args, state, **kwargs):
        result, new_state = self.module(*args, state=state, **kwargs)
        return result


# short for multi-level map
def tree_mlm(fn, tree):
    if isinstance(tree, (int, float, bool, str, np.ndarray, jax.Array, jmp.Policy)):
        return tree
    leaves, treedef = jax.tree_util.tree_flatten_with_path(
        tree, is_leaf=lambda x: x is not tree
    )
    if not leaves:
        return tree
    if any(x is tree for x in leaves):
        return
    leaves = [fn(path, tree_mlm(fn, leaf)) for path, leaf in leaves]
    return jax.tree_util.tree_unflatten(treedef, leaves)


def debugify_model(model):
    def new_model(*args, **kwargs):
        logs = []

        def debugify(path, leaf):
            if not isinstance(leaf, eqx.Module):
                return leaf
            if hasattr(leaf, "__call__"):
                call = leaf.__call__
                if hasattr(call, "_orig_fn"):
                    leaf = eqx.tree_at(lambda l: l.__call__, leaf, call._orig_fn)
                call_ = add_state(call)
                call_._orig_fn = call
                call = call_
                call.store = lambda state, output: (
                    output,
                    {"".join(map(str, path)): output, **state},
                )
                print(leaf)
                eqx.tree_at(lambda l: l.__call__, leaf, call)

        return tree_mlm(debugify, model)(*args, **kwargs), dict(logs)

    return new_model


llama_debug = jax.vmap(debugify_model(llama))
shmoogle_smi()

with mesh:
    input_ids = [
        tokenizer.encode(x, bos=True, eos=False)
        for x in ["Hello world", "This is a test"]
    ]
    input_ids = [x + [0] * (128 - len(x)) for x in input_ids]
    ids = jnp.asarray(input_ids)
    ids = jax.device_put(ids, NamedSharding(mesh, spec=PartitionSpec("dp", None)))
    result = llama_debug(ids)

Total mem usage: 30.0 GB out of 128 GB
Module(body=[If(test=Constant(value=True, kind=None), body=[FunctionDef(name='__call__', args=arguments(posonlyargs=[], args=[arg(arg='self', annotation=None, type_comment=None), arg(arg='x', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[arg(arg='state', annotation=None, type_comment=None)], kw_defaults=[Dict(keys=[], values=[])], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='x', ctx=Store())], value=Subscript(value=NamedExpr(target=Name(id='state', ctx=Store()), value=Call(func=Call(func=Name(id='getattr', ctx=Load()), args=[Attribute(value=Attribute(value=Name(id='self', ctx=Load()), attr='policy', ctx=Load()), attr='cast_to_compute', ctx=Load()), Constant(value='store', kind=None), Lambda(args=arguments(posonlyargs=[], args=[arg(arg='state', annotation=None, type_comment=None), arg(arg='output', annotation=None, type_comment=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=Tuple(elts

Main binary filename not available.


AssertionError: 

In [12]:
reference_llama = transformers.LlamaModel.from_pretrained("models/Llama-2-7b-hf")

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.62s/it]


In [13]:
import torch


def make_torch_hook(name):
    def torch_hook(module, input, output):
        print(f"Module called: {name} {module.__class__}")
        for arg in input:
            if isinstance(arg, torch.Tensor):
                print("Input:", arg.shape, arg.dtype, str(arg)[:100])
        if isinstance(output, tuple):
            output = output[0]
        if isinstance(output, torch.Tensor):
            print("Output:", output.shape, output.dtype, str(output)[:100])
        return output

    return torch_hook


for name, module in reference_llama.named_modules():
    module._forward_hooks.clear()
    module.register_forward_hook(make_torch_hook(name))


input_ids = torch.tensor(input_ids)
reference_llama(input_ids)

Module called: embed_tokens <class 'torch.nn.modules.sparse.Embedding'>
Input: torch.Size([2, 128]) torch.int64 tensor([[    1, 15043,  3186,     0,     0,     0,     0,     0,     0,     0,
             0,     0
Output: torch.Size([2, 128, 4096]) torch.float32 tensor([[[ 1.8616e-03, -3.3722e-03,  3.9864e-04,  ..., -8.3008e-03,
           2.5787e-03, -3.9368e-
Module called: layers.0.input_layernorm <class 'transformers.models.llama.modeling_llama.LlamaRMSNorm'>
Input: torch.Size([2, 128, 4096]) torch.float32 tensor([[[ 1.8616e-03, -3.3722e-03,  3.9864e-04,  ..., -8.3008e-03,
           2.5787e-03, -3.9368e-
Output: torch.Size([2, 128, 4096]) torch.float32 tensor([[[ 6.7356e-03, -5.5986e-03,  9.5712e-05,  ..., -1.0382e-02,
           3.4557e-03, -2.9162e-
Module called: layers.0.self_attn.q_proj <class 'torch.nn.modules.linear.Linear'>
Input: torch.Size([2, 128, 4096]) torch.float32 tensor([[[ 6.7356e-03, -5.5986e-03,  9.5712e-05,  ..., -1.0382e-02,
           3.4557e-03, -2.9162e-
Out

ValueError: too many values to unpack (expected 2)